In [1]:
%pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 37.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import shutil

def convert_annotations_to_yolo(json_file, images_dir, output_dir):
    """
    Mengonversi anotasi Pascal VOC ke format YOLO.
    Format YOLO: class x_center y_center width height (semua dinormalisasi 0-1)
    """
    with open(json_file, 'r') as f:
        annotations = json.load(f)

    os.makedirs(output_dir, exist_ok=True)
    labels_dir = os.path.join(output_dir, "labels")
    images_output_dir = os.path.join(output_dir, "images")

    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(images_output_dir, exist_ok=True)

    for annotation in annotations:
        image_id = annotation['image_id']
        bbox = annotation['bbox']  # Pascal VOC: [xmin, ymin, xmax, ymax]
        category_id = annotation['category_id'] - 1  # YOLO classes start from 0

        # Convert bbox to YOLO format
        x_center = (bbox[0] + bbox[2]) / 2 / 640  # Normalize with image width
        y_center = (bbox[1] + bbox[3]) / 2 / 640
        width = (bbox[2] - bbox[0]) / 640
        height = (bbox[3] - bbox[1]) / 640

        # Write label file
        label_file = os.path.join(labels_dir, f"{image_id}.txt")
        with open(label_file, 'w') as f_out:
            f_out.write(f"{category_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        # Copy images
        shutil.copy(os.path.join(images_dir, f"{image_id}.jpg"), images_output_dir)

# Convert train and validation annotations
convert_annotations_to_yolo("data/train_annotations.json", "data/train", "data/yolo_train")
convert_annotations_to_yolo("data/valid_annotations.json", "data/valid", "data/yolo_valid")
